# Project: Objects and Classes

## Problem

Design and implement a class that will be used to represent bank accounts.
The class should consist of following functionality:

- Accounts are uniquely identified by an account number.(Assume it will be just passed in the initializer)
- Account holders have "first" and "last" name.
- "Balances" need to be zero or higher, and should not be directly settable.
- "Deposits" and "Withdrawals"can be made (given sufficient funds)
    - If a withdrawal is attempted that would result in negative funds, the transaction should be declined.
- A "monthly interest rate" exists and is applicable to all accounts "uniformly". There should be a method that can be called to calculate the interest on the current balance using the current interest rate, and "add" to the balance.
- Each deposit and withdrawal must generate a "confirmation number" composed of it:
    - The transaction type: `D` for deposit, `W` for withdrawal, `I` for interest deposit, `X` for declined(in this case balance remains unaffected)
    - Account number.
    - The time the transaction was made, using UTC.
    - An incrementing number (that increments across all accounts and transactions)
    - for (extreme) simplicity assume that the transaction id starts at zero (or whatever number you choose) whenever the program starts.
    the confirmation number should be returned from any of the transaction methods (deposit, withdraw, etc.)
- Create a **method** that, given a confirmation number, returns:
    - The account number, transaction code (D, W, etc.), datetime (UTC format), data time (in whatever timezone is specified in the argument, but more human-readable), the transaction ID.
    - make it so it is a nice structure object (so can use dot notation to access these three attributes)


## Example:

- Account number: `31415926`
- preferred time zone: +5:30 (IST)
- An existing balance of `100.0`

Suppose the last transaction ID in the system was `123`, and a deposit is made for `50.0` on `2023-08-30T13:57:00` (UTC) on that account or `2023-08-30T19:27:00` (IST).

The new balance should reflect `150.00` and the confirmation number returned should look something like this:

`D-31415926-20230830135700-124`

The requirement is also a method that given the confirmation number returns an object with attributes:
    - `result.account_number` -> `31415926`
    - `result.transaction_code` -> `D`
    - `result.transaction_id` -> `124`
    - `result.time` -> `2023-08-30 19:27:00 (MST)`
    - `result.time_utc` -> `2023-08-30T13:57:00`
    
- Furthermore, if current interest rate is `0.5%` and the account's balance is `1000.00` then the result of calling the `deposit_interest`(or whatever name you select for that method) should result in a new balance of `1005.00`. Calling this method should also return a confirmation number.


## First Thought Process

- Create two classes: First, `Timezone` class used to store the time zone name and offset definition(in hours and minutes). Second, `Account` that will have following "public" interface:
    - initializer with account number, first name, last name, optional preferred time zone, starting balance.
    - a first name property (read/write)
    - a last name property (read/write)
    - a full name property (computed, read-only)
    - a balance property (read-only)
    - an interest rate property (class level property)
    - deposit, withdraw, pay_interest methods
    - parse confirmation code
    
    

## Timezone Class

In [3]:
import numbers
from datetime import timedelta, datetime

class TimeZone:
    def __init__(self, name, offset_hours, offset_minutes):
        """
        
        :param name: 
        :param offset_hours: 
        :param offset_minutes: 
        """
        if name is None or len(str(name).strip()) == 0:
            raise ValueError('Timezone name cannot be empty')
        # Stringify and normalize the input data
        self._name = str(name).strip()
        
        if not isinstance(offset_hours, numbers.Integral):
            raise ValueError('Hour offset must be an integer.')
        
        if not isinstance(offset_minutes, numbers.Integral):
            raise ValueError('Minute offset must be an integer')
        
        # Below check can also be performed using absolute value
        if offset_minutes > 59 or offset_minutes < -59:
            raise ValueError('Minutes offset must be between -59 and 59 (inclusive)')
        
        offset = timedelta(hours=offset_hours, minutes=offset_minutes)
        
        if offset < timedelta(hours=-12, minutes=0) or offset > timedelta(hours=14, minutes=0):
            raise ValueError('Offset must be between -12:00 and +14:00.')
        
        self._offset_hours = offset_hours
        self._offset_minutes = offset_minutes
        self._offset = offset
        
        
    @property
    def offset(self):
        return self._offset
    
    @property
    def name(self):
        return self._name
    
    def __eq__(self, other):
        return (isinstance(other, TimeZone) and 
                self.name == other.name and 
                self._offset_hours == other._offset_hours and
                self._offset_minutes == other.__offset_minutes)
    
    def __repr__(self):
        return (f"TimeZone(name='{self.name}', "
                f"offset_hours={self._offset_hours}, "
                f"offset_minutes={self._offset_minutes})")

In [2]:
# Test the class
tz1 = TimeZone('ABC', -2, -15)
tz1.name

'ABC'

In [5]:
dt = datetime.utcnow()
print(dt)
print(dt + tz1.offset)

2023-09-11 19:23:55.286348
2023-09-11 17:08:55.286348


## Transaction Numbers

In [7]:
# Little inefficient way
class TransactionID:
    def __init__(self, start_id):
        self._start_id = start_id
        
    def next(self):
        self._start_id += 1
        return self._start_id
    
class Account:
    def __init__(self):
        self.transaction_counter = TransactionID(100)
    
    def make_transaction(self):
        new_trans_id = self.transaction_counter.next()
        return new_trans_id
    
a1 = Account()
a2 = Account()

print(a1.make_transaction())
print(a2.make_transaction())
print(a1.make_transaction())

101
101
102


In [10]:
# Create a Generator
def transaction_ids(start_id):
    while True:
        start_id += 1
        yield start_id
        
t = transaction_ids(100)
print(t)
print(next(t))
print(next(t))
print(next(t))

<generator object transaction_ids at 0x0000022F27D1A5E0>
101
102
103


In [12]:
# Use a Generator
class Account:
    def __init__(self):
        self.transaction_counter = transaction_ids(100)
        
    def make_transaction(self):
        new_trans_id = next(self.transaction_counter)
        return new_trans_id
    
a1 = Account()
a2 = Account()

print(a1.make_transaction())
print(a2.make_transaction())
print(a1.make_transaction())

101
101
102


In [11]:
# Most efficient way
import itertools

class Account:
    def __init__(self):
        self.transaction_counter = itertools.count(100)
        
    def make_transaction(self): 
        new_trans_id = next(self.transaction_counter)
        return new_trans_id
    
a1 = Account()
a2 = Account()

print(a1.make_transaction())
print(a2.make_transaction())
print(a1.make_transaction())

100
100
101


## Account Number, Names

In [13]:
class Account:
    
    def __init__(self, account_number, first_name, last_name):
        self.transaction_counter = itertools.count(100)
        self._account_number = account_number
        self.first_name = first_name
        self.last_name = last_name
        
    @property
    def account_number(self):
        return self._account_number
    
    @property
    def first_name(self):
        return self._first_name
    
    @first_name.setter
    def first_name(self, value):
        self._first_name = Account.validate_name(value, 'First Name')
        
    @property
    def last_name(self):
        return self._last_name
    
    @last_name.setter
    def last_name(self, value):
        self._last_name = Account.validate_name(value, 'Last Name')
    
    @staticmethod    
    def validate_name(value, field_title):
        if value is None or len(str(value).strip()) == 0:
        # if len(str(value).strip()) == 0:
            raise ValueError(f'{field_title} cannot be empty')
        return str(value).strip()

In [14]:
# Test class
try:
    a = Account('12345', 'John', '')
except ValueError as ex:
    print(ex)

Last Name cannot be empty


## Add preferred timezone

In [19]:
class Account:
    
    def __init__(self, account_number, first_name, last_name, timezone=None):
        self.transaction_counter = itertools.count(100)
        self._account_number = account_number
        self.first_name = first_name
        self.last_name = last_name
        
        if timezone is None:
            timezone = TimeZone('UTC', 0, 0)
        self.timezone = timezone
        
    @property
    def account_number(self):
        return self._account_number
    
    @property
    def first_name(self):
        return self._first_name
    
    @first_name.setter
    def first_name(self, value):
        self._first_name = Account.validate_name(value, 'First Name')
        
    @property
    def last_name(self):
        return self._last_name
    
    @last_name.setter
    def last_name(self, value):
        self._last_name = Account.validate_name(value, 'Last Name')
        
    @property
    def full_name(self):
        return f'{self.first_name} {self.last_name}'
    
    @property
    def timezone(self):
        return self._timezone
    
    @timezone.setter
    def timezone(self, value):
        if not isinstance(value, TimeZone):
            raise ValueError('Time Zone must be a valid TimeZone object')
        self._timezone = value
    
    @staticmethod    
    def validate_name(value, field_title):
        if value is None or len(str(value).strip()) == 0:
        # if len(str(value).strip()) == 0:
            raise ValueError(f'{field_title} cannot be empty')
        return str(value).strip()

## Account Balance

- Balance needs to be read-only property.
- It should not be directly modified. It should be modified through transactions only.
- Currently there is no any logging, but in production system it should be logged.

In [20]:
class Account:
    
    def __init__(self, 
                 account_number, 
                 first_name, 
                 last_name, 
                 timezone=None, 
                 initial_balance=0):
        """
        
        :param account_number: 
        :param first_name: 
        :param last_name: 
        :param timezone: 
        :param initial_balance: 
        """
        self.transaction_counter = itertools.count(100)
        self._account_number = account_number
        self.first_name = first_name
        self.last_name = last_name
        
        if timezone is None:
            timezone = TimeZone('UTC', 0, 0)
        self.timezone = timezone
        
        self._balance = float(initial_balance)
        
    @property
    def account_number(self):
        return self._account_number
    
    @property
    def first_name(self):
        return self._first_name
    
    @first_name.setter
    def first_name(self, value):
        self._first_name = Account.validate_name(value, 'First Name')
        
    @property
    def last_name(self):
        return self._last_name
    
    @last_name.setter
    def last_name(self, value):
        self._last_name = Account.validate_name(value, 'Last Name')
        
    @property
    def full_name(self):
        return f'{self.first_name} {self.last_name}'
    
    @property
    def balance(self):
        return self._balance
    
    @property
    def timezone(self):
        return self._timezone
    
    @timezone.setter
    def timezone(self, value):
        if not isinstance(value, TimeZone):
            raise ValueError('Time Zone must be a valid TimeZone object')
        self._timezone = value
    
    @staticmethod    
    def validate_name(value, field_title):
        if value is None or len(str(value).strip()) == 0:
        # if len(str(value).strip()) == 0:
            raise ValueError(f'{field_title} cannot be empty')
        return str(value).strip()

In [21]:
a = Account('12345', 'Himanshu', 'Himanshu', initial_balance=100)
a.balance

100.0

In [22]:
try:
    a.balance = 200
except AttributeError as ex:
    print(ex)

can't set attribute 'balance'


## Interest Rate

- Interest rate is common to all bank accounts.
- So, it is not an instance variable but a class variable(attribute or property)

In [23]:
class Account:
    
    interest_rate = 0.5 # Percent
    
    def __init__(self, 
                 account_number, 
                 first_name, 
                 last_name, 
                 timezone=None, 
                 initial_balance=0):
        """
        
        :param account_number: 
        :param first_name: 
        :param last_name: 
        :param timezone: 
        :param initial_balance: 
        """
        self.transaction_counter = itertools.count(100)
        self._account_number = account_number
        self.first_name = first_name
        self.last_name = last_name
        
        if timezone is None:
            timezone = TimeZone('UTC', 0, 0)
        self.timezone = timezone
        
        self._balance = float(initial_balance)
        
    @property
    def account_number(self):
        return self._account_number
    
    @property
    def first_name(self):
        return self._first_name
    
    @first_name.setter
    def first_name(self, value):
        self._first_name = Account.validate_name(value, 'First Name')
        
    @property
    def last_name(self):
        return self._last_name
    
    @last_name.setter
    def last_name(self, value):
        self._last_name = Account.validate_name(value, 'Last Name')
        
    @property
    def full_name(self):
        return f'{self.first_name} {self.last_name}'
    
    @property
    def balance(self):
        return self._balance
    
    @property
    def timezone(self):
        return self._timezone
    
    @timezone.setter
    def timezone(self, value):
        if not isinstance(value, TimeZone):
            raise ValueError('Time Zone must be a valid TimeZone object')
        self._timezone = value
    
    @staticmethod    
    def validate_name(value, field_title):
        if value is None or len(str(value).strip()) == 0:
        # if len(str(value).strip()) == 0:
            raise ValueError(f'{field_title} cannot be empty')
        return str(value).strip()

In [24]:
a1 = Account(1234, 'Monty', 'Python')
a2 = Account(2345, 'John', 'Nash')
a1.interest_rate, a2.interest_rate

(0.5, 0.5)

In [25]:
# Changing the interest rate
Account.interest_rate = 10
a1.interest_rate, a2.interest_rate

(10, 10)

In [26]:
a1.interest_rate = 100
a1.interest_rate, a2.interest_rate

(100, 10)

In [27]:
# Fixing the issue
class Account:
    
    _interest_rate = 0.5 # Percent
    
    def __init__(self, 
                 account_number, 
                 first_name, 
                 last_name, 
                 timezone=None, 
                 initial_balance=0):
        """
        
        :param account_number: 
        :param first_name: 
        :param last_name: 
        :param timezone: 
        :param initial_balance: 
        """
        self.transaction_counter = itertools.count(100)
        self._account_number = account_number
        self.first_name = first_name
        self.last_name = last_name
        
        if timezone is None:
            timezone = TimeZone('UTC', 0, 0)
        self.timezone = timezone
        
        self._balance = float(initial_balance)
        
    @property
    def account_number(self):
        return self._account_number
    
    @property
    def first_name(self):
        return self._first_name
    
    @first_name.setter
    def first_name(self, value):
        self._first_name = Account.validate_name(value, 'First Name')
        
    @property
    def last_name(self):
        return self._last_name
    
    @last_name.setter
    def last_name(self, value):
        self._last_name = Account.validate_name(value, 'Last Name')
        
    @property
    def full_name(self):
        return f'{self.first_name} {self.last_name}'
    
    @property
    def balance(self):
        return self._balance
    
    @property
    def timezone(self):
        return self._timezone
    
    @timezone.setter
    def timezone(self, value):
        if not isinstance(value, TimeZone):
            raise ValueError('Time Zone must be a valid TimeZone object')
        self._timezone = value
        
    @classmethod
    def get_interest_rate(cls):
        return cls._interest_rate
    
    @classmethod
    def set_interest_rate(cls, value):
        if not isinstance(value, numbers.Real):
            raise ValueError('Interest rate must be a real number.')
        
        if value < 0:
            raise ValueError('Interest rate cannot be negative.')
        cls._interest_rate = value
    
    @staticmethod    
    def validate_name(value, field_title):
        if value is None or len(str(value).strip()) == 0:
        # if len(str(value).strip()) == 0:
            raise ValueError(f'{field_title} cannot be empty')
        return str(value).strip()

In [28]:
Account.get_interest_rate()

0.5

In [29]:
Account.set_interest_rate(10)
Account.get_interest_rate()

10

## Confirmation codes

- the transaction code
- the account number
- the data/time in UTC of the transaction
- the transaction number

In [30]:
def generate_confirmation_code(account_number, transaction_id, transaction_code):
    dt_string = datetime.utcnow().strftime('%Y%m%d%H%M%S')
    return f'{transaction_code}-{account_number}-{dt_string}-{transaction_id}'

generate_confirmation_code(123, 1000, 'X')

'X-123-20230911212130-1000'

In [31]:
from collections import namedtuple

Confirmation = namedtuple('Confirmation', 'account_number transaction_code transaction_id time_utc time')

In [32]:
class Account:
    
    transaction_counter = itertools.count(100)    
    _interest_rate = 0.5 # Percent
    _transaction_codes = {
        'deposit': 'D', 
        'withdraw': 'W', 
        'interest': 'I', 
        'rejected': 'X'
    }
    def __init__(self, 
                 account_number, 
                 first_name, 
                 last_name, 
                 timezone=None, 
                 initial_balance=0):
        """
        
        :param account_number: 
        :param first_name: 
        :param last_name: 
        :param timezone: 
        :param initial_balance: 
        """
        self._account_number = account_number
        self.first_name = first_name
        self.last_name = last_name
        
        if timezone is None:
            timezone = TimeZone('UTC', 0, 0)
        self.timezone = timezone
        
        self._balance = float(initial_balance)
        
    @property
    def account_number(self):
        return self._account_number
    
    @property
    def first_name(self):
        return self._first_name
    
    @first_name.setter
    def first_name(self, value):
        self._first_name = Account.validate_name(value, 'First Name')
        
    @property
    def last_name(self):
        return self._last_name
    
    @last_name.setter
    def last_name(self, value):
        self._last_name = Account.validate_name(value, 'Last Name')
        
    @property
    def full_name(self):
        return f'{self.first_name} {self.last_name}'
    
    @property
    def balance(self):
        return self._balance
    
    @property
    def timezone(self):
        return self._timezone
    
    @timezone.setter
    def timezone(self, value):
        if not isinstance(value, TimeZone):
            raise ValueError('Time Zone must be a valid TimeZone object')
        self._timezone = value
        
    @classmethod
    def get_interest_rate(cls):
        return cls._interest_rate
    
    @classmethod
    def set_interest_rate(cls, value):
        if not isinstance(value, numbers.Real):
            raise ValueError('Interest rate must be a real number.')
        
        if value < 0:
            raise ValueError('Interest rate cannot be negative.')
        cls._interest_rate = value
        
    def generate_confirmation_code(self, transaction_code):
        dt_string = datetime.utcnow().strftime('%Y%m%d%H%M%S')
        return f'{transaction_code}-{self.account_number}-{dt_string}-{next(Account.transaction_counter)}'
    
    @staticmethod    
    def validate_name(value, field_title):
        if value is None or len(str(value).strip()) == 0:
        # if len(str(value).strip()) == 0:
            raise ValueError(f'{field_title} cannot be empty')
        return str(value).strip()
    
    @staticmethod
    def parse_confirmation_code(confirmation_code, preferred_time_zone=None):
        parts = confirmation_code.split('-')
        if len(parts) != 4:
            raise ValueError('Invalid confirmation code')
        transaction_code, account_number, raw_dt_utc, transaction_id = parts
        try:
            dt_utc = datetime.strptime(raw_dt_utc, '%Y%m%d%H%M%S')
        except ValueError as ex:
            raise ValueError('Invalid transaction datetime') from ex
        
        if preferred_time_zone is None:
            preferred_time_zone = TimeZone('UTC', 0, 0)
            
        if not isinstance(preferred_time_zone, TimeZone):
            raise ValueError('Invalid TimeZone specified')
        
        dt_preferred = dt_utc + preferred_time_zone.offset
        dt_preferred_str = f"{dt_preferred.strftime('%Y-%m-%d %H:%M:%S')} ({preferred_time_zone})"
        
        return Confirmation(account_number, transaction_code, transaction_id, dt_utc.isoformat(), dt_preferred_str)

In [ ]:
a = Account('A1000', 'Harry', 'Potter')
conf_code = a.mak

## Generated by Google Bard

In [ ]:
class BankAccount:
    """
    A class to represent a bank account.

    Attributes:
        account_number: The account number.
        first_name: The account holder's first name.
        last_name: The account holder's last name.
        balance: The account balance.
        interest_rate: The monthly interest rate.
        transaction_id: The transaction ID.

    Methods:
        deposit: Deposits an amount to the account.
        withdraw: Withdraws an amount from the account.
        calculate_interest: Calculates and adds interest to the balance.
        get_confirmation_number: Generates a confirmation number for a transaction.
        get_transaction_details: Returns details of a transaction.
    """

    def __init__(self, account_number, first_name, last_name, balance=0, interest_rate=0.0):
        """
        Initializes a bank account.

        Args:
            account_number: The account number.
            first_name: The account holder's first name.
            last_name: The account holder's last name.
            balance: The account balance.
            interest_rate: The monthly interest rate.
        """
        self.account_number = account_number
        self.first_name = first_name
        self.last_name = last_name
        self.balance = balance
        self.interest_rate = interest_rate
        self.transaction_id = 0

    def deposit(self, amount):
        """
        Deposits an amount to the account.

        Args:
            amount: The amount to deposit.

        Returns:
            The confirmation number for the transaction.
        """
        if amount < 0:
            raise ValueError("Amount must be non-negative.")

        self.balance += amount
        self.transaction_id += 1
        return f"D{self.account_number}{datetime.utcnow()}:{self.transaction_id}"

    def withdraw(self, amount):
        """
        Withdraws an amount from the account.

        Args:
            amount: The amount to withdraw.

        Returns:
            The confirmation number for the transaction.
        """
        if amount < 0:
            raise ValueError("Amount must be non-negative.")

        if amount > self.balance:
            return "X"

        self.balance -= amount
        self.transaction_id += 1
        return f"W{self.account_number}{datetime.utcnow()}:{self.transaction_id}"

    def calculate_interest(self):
        """
        Calculates and adds interest to the balance.
        """
        interest = self.balance * self.interest_rate
        self.balance += interest

    def get_confirmation_number(self, transaction_type):
        """
        Generates a confirmation number for a transaction.

        Args:
            transaction_type: The type of transaction.

        Returns:
            The confirmation number.
        """
        return f"{transaction_type}{self.account_number}{datetime.utcnow()}:{self.transaction_id}"

    def get_transaction_details(self, confirmation_number):
        """
        Returns details of a transaction.

        Args:
            confirmation_number: The confirmation number.

        Returns:
            A dictionary containing the transaction details.
        """
        transaction_type, account_number, timestamp, transaction_id = confirmation_number.split(":")
        transaction_date = datetime.strptime(timestamp, "%Y-%m-%dT%H:%M:%S.%f")
        return {
            "transaction_type": transaction_type,
            "account_number": account_number,
            "transaction_date": transaction_date,
            "transaction_id": transaction_id,
        }
